In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import pytorch_lightning as pl
import pandas as pd
import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
# Utils
import matplotlib.pyplot as plt
import numpy as np

# save checkpoint
# Function to save model checkpoint
def save_checkpoint(model, scaler, path='model_checkpoint.pth'):
    checkpoint = {
        'state_dict': model.state_dict(),
        'threshold': model.threshold,
        'scaler': scaler
    }
    torch.save(checkpoint, path)
    print(f'Checkpoint saved at epoch')

def load_checkpoint(path, model):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['state_dict'])
    model.threshold = checkpoint['threshold']
    scaler = checkpoint['scaler']
    return scaler, model


def plot_time_series(time_series, prediction, title='Time Series with Quantiles'):
    """
    Plots the real time series and quantiles predicted by the autoencoder.

    Parameters:
    - time_series: numpy array of the actual time series values.
    - quantiles: numpy array of shape (n_samples, n_quantiles, seq_length) containing the predicted quantiles.
    - seq_length: int, length of the sequence used for the predictions.
    - title: str, title of the plot.
    """
    
    # Plot the actual time series
    plt.figure(figsize=(14, 8))
    plt.plot(time_series, label='Actual Time Series', color='blue')
    plt.plot(prediction, label='Predition', color='orange')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

def plot_time_series_with_errors_area(time_series, prediction, error_low, error_high):
    """
    Plots the real time series and quantiles predicted by the autoencoder.

    Parameters:
    - time_series: numpy array of the actual time series values.
    - quantiles: numpy array of shape (n_samples, n_quantiles, seq_length) containing the predicted quantiles.
    - seq_length: int, length of the sequence used for the predictions.
    - title: str, title of the plot.
    """
    
    # Plot the actual time series
    plt.figure(figsize=(14, 8))
    plt.plot(time_series, label='Actual Time Series', color='blue')
    plt.plot(prediction, label='Predition', color='orange')
    plt.fill_between(np.arange(0, len(time_series)), error_low, error_high, alpha=0.3)
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

def plot_time_series_with_quantiles(time_series, quantiles, seq_length, title='Time Series with Quantiles'):
    """
    Plots the real time series and quantiles predicted by the autoencoder.

    Parameters:
    - time_series: numpy array of the actual time series values.
    - quantiles: numpy array of shape (n_samples, n_quantiles, seq_length) containing the predicted quantiles.
    - seq_length: int, length of the sequence used for the predictions.
    - title: str, title of the plot.
    """
    
    # Plot the actual time series
    plt.figure(figsize=(14, 8))
    plt.plot(time_series, label='Actual Time Series', color='blue')
    
    # Plot the quantiles
    for index in range(0, len(quantiles)//2):
        print(quantiles[index])
        print(quantiles[len(quantiles)-index-1])
        plt.fill_between(np.arange(0, len(time_series)), 
                         quantiles[index], quantiles[len(quantiles)-index-1], label=f'Quantile {index+1}', alpha=0.3)
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.show()



In [3]:
def get_scaler(values):
    values = values.reshape(-1, 1)
    scaler = StandardScaler()
    scaler = scaler.fit(values)
    return scaler


def create_dataloader(values, scaler, seq_length, train_ratio):
    values = values.reshape(-1, 1)

    # Normalize the data
    values_scaled = scaler.transform(values)

    # Create sequences
    def create_sequences(data, seq_length):
        sequences = []
        for i in range(len(data) - seq_length):
            sequences.append(data[i:i + seq_length])
        return np.array(sequences)

    X = create_sequences(values_scaled, seq_length)
    X = torch.tensor(X, dtype=torch.float32)
    dataset = TensorDataset(X, X)

    train_size = int(train_ratio * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)
    return dataset, train_dataloader, val_dataloader

In [4]:
throughput = pd.read_csv('./../data/throughput.csv')
web_response = pd.read_csv('./../data/web_response.csv')
apdex = pd.read_csv('./../data/apdex.csv')

Comvolutional

In [58]:
class ConvAutoencoder(pl.LightningModule):
    def __init__(self, loss, hidden_size, seq_len):
        super(ConvAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 32, 3, padding=1),  # Output: [batch_size, 32, seq_len]
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2),       # Output: [batch_size, 32, seq_len//2]
            nn.Conv1d(32, 16, 3, padding=1), # Output: [batch_size, 16, seq_len//2]
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2),       # Output: [batch_size, 16, seq_len//4]
            nn.Conv1d(16, 8, 3, padding=1),  # Output: [batch_size, 8, seq_len//4]
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2)        # Output: [batch_size, 8, seq_len//8]
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(8, 16, 4, stride=2, padding=1), # Output: [batch_size, 16, seq_len//4]
            nn.ReLU(),
            nn.ConvTranspose1d(16, 32, 4, stride=2, padding=1), # Output: [batch_size, 32, seq_len//2]
            nn.ReLU(),
            nn.ConvTranspose1d(32, 1, 4, stride=2, padding=1),  # Output: [batch_size, 1, seq_len]
        )
        self.loss = loss
        self.seq_len = seq_len
    
    def forward(self, x):
        x = x.permute(0, 2, 1)  # [batch_size, seq_len, 1] -> [batch_size, 1, seq_len]
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        decoded = decoded.permute(0, 2, 1)  # [batch_size, 1, seq_len] -> [batch_size, seq_len, 1]
        return decoded
    
    def training_step(self, batch, batch_idx):
        inputs, _ = batch
        outputs = self(inputs)
        loss = self.loss(outputs, inputs.permute(0, 2, 1))
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        return optimizer
    
    # Function to evaluate model on validation set
    def evaluate_threshhold(self, val_loader, percentile):
        self.eval()  # Set model to evaluation mode
        val_loss = []
        criterion = nn.L1Loss()
        with torch.no_grad():  # Disable gradient calculation
            for x, y in val_loader:
                y_hat = self(x)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
        THRESHOLD = np.percentile(val_loss, percentile)
        self.threshold = THRESHOLD
        return THRESHOLD

    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)


In [59]:
scaler = get_scaler(throughput['sum_call_count'].values)
dataset, train_dataloader, val_dataloader = create_dataloader(throughput['sum_call_count'].values, scaler, 64, 0.8)

In [60]:
# Instantiate the model
model = ConvAutoencoder(hidden_size=128, loss=nn.L1Loss(), seq_len=64)
# Initialize EarlyStopping callback
early_stop_callback = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=10,          # Number of epochs with no improvement to stop training
    verbose=True,        # Enable verbose mode
    mode='min'           # Mode 'min' to stop when the metric stops decreasing
)
# Initialize ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',      # Metric to monitor
    dirpath='checkpoints/',  # Directory to save checkpoints
    filename='best-checkpoint',  # Filename for the best checkpoint
    save_top_k=1,            # Save only the best checkpoint
    mode='min',              # Mode 'min' to save the checkpoint with the lowest 'val_loss'
    save_last=True,          # Save the final model
    verbose=True
)
checkpoint_callback = ModelCheckpoint('./')
trainer = pl.Trainer(max_epochs=2, logger=True, log_every_n_steps=1, val_check_interval=500, callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


d:\courses\MLsimulator\venv\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:653: Checkpoint directory D:\projects\code\red-hack-time-series-analyzer\notebooks exists and is not empty.

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 2.1 K 
1 | decoder | Sequential | 2.7 K 
2 | loss    | L1Loss     | 0     
---------------------------------------
4.8 K     Trainable params
0         Non-trainable params
4.8 K     Total params
0.019     Total estimated model params size (MB)


d:\courses\MLsimulator\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
d:\courses\MLsimulator\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
d:\courses\MLsimulator\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0:   1%|          | 6/1081 [00:00<00:21, 49.89it/s, v_num=59, train_loss_step=0.886]

d:\courses\MLsimulator\venv\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([32, 1, 64])) that is different to the input size (torch.Size([32, 64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


...

Epoch 1:   1%|          | 8/1081 [00:00<00:17, 59.76it/s, v_num=59, train_loss_step=0.229, val_loss_step=0.0899, val_loss_epoch=0.168, train_loss_epoch=0.264]   

d:\courses\MLsimulator\venv\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([6, 1, 64])) that is different to the input size (torch.Size([6, 64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 1: 100%|██████████| 1081/1081 [00:27<00:00, 40.01it/s, v_num=59, train_loss_step=0.171, val_loss_step=0.168, val_loss_epoch=0.162, train_loss_epoch=0.232]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 1081/1081 [00:27<00:00, 40.00it/s, v_num=59, train_loss_step=0.171, val_loss_step=0.168, val_loss_epoch=0.162, train_loss_epoch=0.232]


LSTM TRAINING

In [214]:
import torch
import torch.nn as nn

class LSTMModel(pl.LightningModule):
    def __init__(self, input_dim, seq_len, hidden_dim=128, dropout_rate=0.2, loss=nn.L1Loss()):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
        self.input_dim = input_dim
        
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.repeat_vector = lambda x: x.unsqueeze(1).repeat(1, self.seq_len, 1)
        
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.time_distributed_dense = nn.Linear(hidden_dim, input_dim)
        self.loss = loss

    def forward(self, x):
        # LSTM layer
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        
        # RepeatVector
        x = self.repeat_vector(x[:, -1, :])
        
        # LSTM layer
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        
        # TimeDistributed Dense layer
        x = self.time_distributed_dense(x)
        return x
    
    
    def training_step(self, batch, batch_idx):
        x, _ = batch
        y_pred = self(x)
        loss = self.loss(y_pred, x)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        return optimizer
    
    # Function to evaluate model on validation set
    def evaluate_threshhold(self, val_loader, percentile):
        self.eval()  # Set model to evaluation mode
        val_loss = []
        criterion = nn.L1Loss()
        with torch.no_grad():  # Disable gradient calculation
            for x, y in val_loader:
                y_hat = self(x)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
        THRESHOLD = np.percentile(val_loss, percentile)
        self.threshold = THRESHOLD
        return THRESHOLD


array([3898. , 3917.5, 3993. , ..., 2429. , 2373.5, 1170. ])

In [223]:
scaler = get_scaler(throughput['sum_call_count'].values)
dataset, train_dataloader, val_dataloader = create_dataloader(throughput['sum_call_count'].values, scaler, 20, 0.8)

In [225]:
# Instantiate the model
model = LSTMModel(input_dim=1, hidden_dim=128, seq_len=20)
# Initialize EarlyStopping callback
early_stop_callback = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=0,          # Number of epochs with no improvement to stop training
    verbose=True,        # Enable verbose mode
    mode='min'           # Mode 'min' to stop when the metric stops decreasing
)
# Initialize ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',      # Metric to monitor
    dirpath='checkpoints/',  # Directory to save checkpoints
    filename='best-checkpoint',  # Filename for the best checkpoint
    save_top_k=1,            # Save only the best checkpoint
    mode='min',              # Mode 'min' to save the checkpoint with the lowest 'val_loss'
    save_last=True,          # Save the final model
    verbose=True
)
checkpoint_callback = ModelCheckpoint('./')
trainer = pl.Trainer(max_epochs=2, logger=True, log_every_n_steps=1, val_check_interval=100, callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                   | Type    | Params
---------------------------------------------------
0 | lstm1                  | LSTM    | 67.1 K
1 | dropout1               | Dropout | 0     
2 | lstm2                  | LSTM    | 132 K 
3 | dropout2               | Dropout | 0     
4 | time_distributed_dense | Linear  | 129   
5 | loss                   | L1Loss  | 0     
---------------------------------------------------
199 K     Trainable params
0         Non-trainable params
199 K     Total params
0.797     Total estimated model params size (MB)


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 87.72it/s]

d:\courses\MLsimulator\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0:   9%|▉         | 100/1082 [00:02<00:24, 40.56it/s, v_num=47, train_loss_step=0.237]

Metric val_loss improved. New best score: 0.259


...

Epoch 1:  46%|████▌     | 500/1082 [00:28<00:33, 17.35it/s, v_num=47, train_loss_step=0.166, val_loss_step=0.138, val_loss_epoch=0.139, train_loss_epoch=0.211] 

Monitored metric val_loss did not improve in the last 1 records. Best score: 0.139. Signaling Trainer to stop.


Epoch 1:  46%|████▌     | 500/1082 [00:32<00:38, 15.29it/s, v_num=47, train_loss_step=0.166, val_loss_step=0.121, val_loss_epoch=0.143, train_loss_epoch=0.151]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1:  46%|████▌     | 500/1082 [00:32<00:38, 15.29it/s, v_num=47, train_loss_step=0.166, val_loss_step=0.121, val_loss_epoch=0.143, train_loss_epoch=0.151]


In [190]:
# save checkpoint
# Function to save model checkpoint
def save_checkpoint(model, path='model_checkpoint.pth'):
    checkpoint = {
        'state_dict': model.state_dict(),
        'threshold': model.threshold,
        'scaler': scaler
    }
    torch.save(checkpoint, path)
    print(f'Checkpoint saved at epoch')

def load_checkpoint(path):
    checkpoint = torch.load(path)
    model = LSTMModel(input_dim=1, seq_len=60)
    model.load_state_dict(checkpoint['state_dict'])
    model.threshold = checkpoint['threshold']
    scaler = checkpoint['scaler']
    return scaler, model


In [191]:
save_checkpoint(model, 'model_checkpoint.pth')

Checkpoint saved at epoch


In [192]:
scaler, loaded_model = load_checkpoint('model_checkpoint.pth')

# Visualize model

In [194]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# Assuming x_train and y_train are your data tensors
# Example:
# x_train = torch.randn(1000, 10, 10)  # 1000 samples, sequence length 10, 10 features each
# y_train = torch.randn(1000, 10, 10)  # 1000 samples, sequence length 10, 10 targets each

# Define a function to analyze the time series using the model
def detect_anomalies(model, time_series, threshold, window_size):
    model.eval()  # Set the model to evaluation mode
    anomalies = []
    windows = []
    predicted_values = []
    with torch.no_grad():  # Disable gradient calculation
        for i in range(0, len(time_series) - window_size + 1, window_size):
            window = time_series[i:i + window_size].unsqueeze(0)  # Add batch dimension
            prediction = model(window).squeeze(0)

            windows.extend(window.tolist())
            predicted_values.extend(prediction.tolist())

            # error = torch.abs(prediction - window.squeeze(0))
            # for j in range(window_size):
            #     if error[j].item() > threshold:
            #         anomalies.append(i + j)  # Mark the specific point as an anomaly
    return windows, anomalies, predicted_values

# Example time series data (replace with your actual data)
time_series_data = torch.FloatTensor(scaler.transform(throughput['sum_call_count'].values[0:240].reshape(-1,1)))  # Replace with your actual time series data
window_size = 60  # Replace with your actual window size
threshold = model.threshold  # Replace with your actual threshold for anomaly detection

# Detect anomalies
windows, anomalies, predicted_values = detect_anomalies(model, time_series_data, threshold, window_size)

# # Plot the time series, predicted values, and mark the anomalies
# plt.figure(figsize=(12, 6))
# plt.plot(time_series_data.numpy(), label='Time Series')
# plt.scatter(anomalies, time_series_data[anomalies].numpy(), color='red', label='Anomalies', marker='x')
# plt.xlabel('Time')
# plt.ylabel('Value')
# plt.title('Time Series with Anomalies and Predicted Values')
# plt.legend()
# plt.show()

In [227]:
model(time_series_data[0:20].unsqueeze(0))

tensor([[[-0.0034],
         [-0.0126],
         [-0.0244],
         [-0.0327],
         [-0.0377],
         [-0.0405],
         [-0.0422],
         [-0.0432],
         [-0.0438],
         [-0.0442],
         [-0.0444],
         [-0.0446],
         [-0.0446],
         [-0.0447],
         [-0.0447],
         [-0.0448],
         [-0.0448],
         [-0.0448],
         [-0.0448],
         [-0.0448]]], grad_fn=<ViewBackward0>)

In [228]:
time_series_data[0:20]

tensor([[ 0.2258],
        [ 0.2419],
        [ 0.3043],
        [ 0.3030],
        [ 0.2402],
        [ 0.2249],
        [ 0.0977],
        [ 0.0179],
        [ 0.0622],
        [ 0.0374],
        [-0.1031],
        [-0.1262],
        [-0.1139],
        [-0.1899],
        [-0.2238],
        [-0.2750],
        [-0.2969],
        [-0.3386],
        [-0.3601],
        [-0.2403]])

In [195]:
windows

[[[0.22576750814914703],
  [0.24188075959682465],
  [0.304267942905426],
  [0.3030284643173218],
  [0.24022811651229858],
  [0.2249411791563034],
  [0.09768781810998917],
  [0.017947886139154434],
  [0.062156036496162415],
  [0.03736642003059387],
  [-0.10310807824134827],
  [-0.12624505162239075],
  [-0.11385024338960648],
  [-0.18987172842025757],
  [-0.22375087440013885],
  [-0.274982750415802],
  [-0.29688024520874023],
  [-0.3386094272136688],
  [-0.36009377241134644],
  [-0.24027729034423828],
  [-0.1539267897605896],
  [-0.17541112005710602],
  [-0.22292456030845642],
  [-0.2406904399394989],
  [-0.16053734719753265],
  [-0.0696420893073082],
  [-0.020476019009947777],
  [-0.004362768959254026],
  [0.029516372829675674],
  [0.09768781810998917],
  [0.04934806749224663],
  [0.1745356321334839],
  [0.1823856681585312],
  [-0.003949608653783798],
  [0.0010083146626129746],
  [0.05224018916487694],
  [0.0724850445985794],
  [0.04521646350622177],
  [-0.04981039837002754],
  [-0.1824